In [1]:
repo_path = "/home/kjakkala/mmwave"

import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

import sys
sys.path.append(os.path.join(repo_path, 'models'))

import h5py
from utils import *
from tqdm import tqdm
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

print(tf.__version__)

2.1.0


In [2]:
dataset_path    = os.path.join(repo_path, 'data')
num_classes     = 9
batch_size      = 64
train_src_days  = 3
train_trg_days  = 0
train_trg_env_days = 2
epochs          = 250
init_lr         = 0.0001
num_features    = 256
alpha           = 0.05
clas_hidden     = [64]
activation_fn   = 'selu'
m               = 0.35
s               = 10
notes           = "baseline-s-{}_m-{}-clas-{}".format(s, m, clas_hidden)
log_data = "classes-{}_bs-{}_train_src_days-{}_train_trg_days-{}_train_trgenv_days-{}_alpha-{}_initlr-{}_num_feat-{}_act_fn-{}_{}".format(num_classes,
                                                                                                                                 batch_size,
                                                                                                                                 train_src_days,
                                                                                                                                 train_trg_days,
                                                                                                                                 train_trg_env_days,
                                                                                                                                 alpha,
                                                                                                                                 init_lr,
                                                                                                                                 num_features,
                                                                                                                                 activation_fn,
                                                                                                                                 notes)
log_dir         = os.path.join(repo_path, 'logs/TEST/{}'.format(log_data))
encodings_file  = os.path.join(repo_path, 'data/encodings.h5')

In [3]:
hf = h5py.File(encodings_file, 'r')

X_train_src = np.array(hf.get('X_train_src'))
y_train_src = np.array(hf.get('y_train_src'))
X_test_src  = np.array(hf.get('X_test_src'))
y_test_src  = np.array(hf.get('y_test_src'))
X_train_trg = np.array(hf.get('X_train_trg'))
y_train_trg = np.array(hf.get('y_train_trg'))
X_test_trg  = np.array(hf.get('X_test_trg'))
y_test_trg  = np.array(hf.get('y_test_trg'))

X_train_conf = np.array(hf.get('X_train_conf'))
y_train_conf = np.array(hf.get('y_train_conf'))
X_test_conf  = np.array(hf.get('X_test_conf'))
y_test_conf  = np.array(hf.get('y_test_conf'))

X_train_server = np.array(hf.get('X_train_server'))
y_train_server = np.array(hf.get('y_train_server'))
X_test_server  = np.array(hf.get('X_test_server'))
y_test_server  = np.array(hf.get('y_test_server'))

X_data_office  = np.array(hf.get('X_data_office'))
y_data_office  = np.array(hf.get('y_data_office'))

hf.close()

print("Final shapes: ")
print(X_train_src.shape, y_train_src.shape,  X_test_src.shape, y_test_src.shape, X_train_trg.shape, y_train_trg.shape, X_test_trg.shape, y_test_trg.shape)
print(X_train_conf.shape,   y_train_conf.shape,    X_test_conf.shape,   y_test_conf.shape)
print(X_train_server.shape, y_train_server.shape,  X_test_server.shape, y_test_server.shape)
print(X_data_office.shape,  y_data_office.shape)

Final shapes: 
(2308, 256) (2308, 9) (257, 256) (257, 9) (0,) (0, 9) (5982, 256) (5982, 9)
(0,) (0,) (1350, 256) (1350, 9)
(898, 256) (898, 9) (448, 256) (448, 9)
(899, 256) (899, 9)


In [4]:
def get_target_labels(target_data, target_labels):
    centers = []
    for i in range(num_classes):
        centers.append(X_train_src[np.where(np.argmax(y_train_src, axis=1)==i)].mean(axis=0))
    centers = np.array(centers) 

    kmeans = KMeans(n_clusters=9, init=centers, n_init=1).fit(target_data)
    print((np.argmax(target_labels, axis=1)==kmeans.labels_).mean())
    return tf.one_hot(kmeans.labels_, num_classes)

In [5]:
#get tf.data objects for each set

#Test
conf_test_set = tf.data.Dataset.from_tensor_slices((X_test_conf, y_test_conf))
conf_test_set = conf_test_set.batch(batch_size, drop_remainder=False)
conf_test_set = conf_test_set.prefetch(batch_size)

server_test_set = tf.data.Dataset.from_tensor_slices((X_test_server, y_test_server))
server_test_set = server_test_set.batch(batch_size, drop_remainder=False)
server_test_set = server_test_set.prefetch(batch_size)

office_test_set = tf.data.Dataset.from_tensor_slices((X_data_office, y_data_office))
office_test_set = office_test_set.batch(batch_size, drop_remainder=False)
office_test_set = office_test_set.prefetch(batch_size)

src_test_set = tf.data.Dataset.from_tensor_slices((X_test_src, y_test_src))
src_test_set = src_test_set.batch(batch_size, drop_remainder=False)
src_test_set = src_test_set.prefetch(batch_size)

time_test_set = tf.data.Dataset.from_tensor_slices((X_test_trg, y_test_trg))
time_test_set = time_test_set.batch(batch_size, drop_remainder=False)
time_test_set = time_test_set.prefetch(batch_size)

#Train
src_train_set = tf.data.Dataset.from_tensor_slices((X_train_src, y_train_src))
src_train_set = src_train_set.shuffle(X_train_src.shape[0])
src_train_set = src_train_set.batch(batch_size, drop_remainder=True)
src_train_set = src_train_set.prefetch(batch_size)

server_train_set = tf.data.Dataset.from_tensor_slices((X_train_server, y_train_server, get_target_labels(X_train_server, y_train_server)))
server_train_set = server_train_set.shuffle(X_train_server.shape[0])
server_train_set = server_train_set.batch(batch_size, drop_remainder=True)
server_train_set = server_train_set.prefetch(batch_size)
server_train_set = server_train_set.repeat(-1)

0.9710467706013363


In [6]:
y_soft_train_server = get_target_labels(X_train_server, y_train_server)
y_soft_train_server = np.argmax(y_soft_train_server, axis=-1)

means = []
stds  = []
for i in range(num_classes):
  means.append(np.mean(X_train_server[np.where(y_soft_train_server==i)], axis=0))
  stds.append(np.std(X_train_server[np.where(y_soft_train_server==i)], axis=0)*6)
  
def get_target_mcmc(means, stds, batch_size):
  features = []
  labels = []
  for _ in range(batch_size):
    class_label = np.random.randint(0, num_classes)
    features.append(tf.random.normal([num_features],
                                     mean=means[class_label],
                                     stddev=stds[class_label]))
    labels.append(class_label)
    
  labels = tf.one_hot(labels, num_classes)
  features = tf.stack(features, axis=0)
  return features, labels

0.9710467706013363


In [7]:
class ConstrictiveRegularizer(tf.keras.regularizers.Regularizer):
  def __init__(self, scale):
    super().__init__()
    self.scale = scale

  def __call__(self, x):
    l2_norm = tf.reduce_sum(tf.square(x), axis=0)
    regularization = tf.reduce_mean(l2_norm-tf.reduce_mean(l2_norm))/4.0
    return self.scale * regularization

class AMDense(tf.keras.layers.Layer):
  def __init__(self,
               units,
               kernel_initializer='glorot_uniform',
               kernel_regularizer=None,
               **kwargs):

    super().__init__(**kwargs)
    self.units = units
    self.kernel_initializer = kernel_initializer
    self.kernel_regularizer = kernel_regularizer

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]), self.units],
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  trainable=True)

  def call(self, inputs):
    return tf.matmul(tf.nn.l2_normalize(inputs, -1),
                     tf.nn.l2_normalize(self.kernel, 0))

  
class Classifier(tf.keras.Model):
  def __init__(self, num_hidden, num_classes, activation='relu', ca_decay=1e-3):
    super().__init__(name='classifier')
    self.hidden_layers = []
    for dim in num_hidden[:-1]:
      self.hidden_layers.append(tf.keras.layers.Dense(dim, activation=activation))
    self.hidden_layers.append(tf.keras.layers.Dense(num_hidden[-1], 
                                                    activation=activation, 
                                                    activity_regularizer=ConstrictiveRegularizer(ca_decay)))
    self.logits = AMDense(num_classes,
                          kernel_regularizer=ConstrictiveRegularizer(ca_decay),
                          name='logits')
    
  def call(self, x):
    for layer in self.hidden_layers:
      x = layer(x)
    embed = x
    x = self.logits(x)
    return x, embed
  
class MLP(tf.keras.Model):
  def __init__(self, num_hidden, activation='relu'):
    super().__init__(name='mlp')
    self.hidden_layers = []
    for dim in num_hidden[:-1]:
      self.hidden_layers.append(tf.keras.layers.Dense(dim, activation=activation))
    self.hidden_layers.append(tf.keras.layers.Dense(num_hidden[-1], activation=None))
      
  def call(self, x):
    for layer in self.hidden_layers:
      x = layer(x)
    return x
  
def get_cross_entropy_loss(labels, logits):
  loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
  return tf.reduce_mean(loss)

def AM_logits(labels, logits, m, s):
  cos_theta = tf.clip_by_value(logits, -1,1)
  phi = cos_theta - m
  adjust_theta = s * tf.where(tf.equal(labels,1), phi, cos_theta)
  return adjust_theta

In [8]:
cross_entropy_loss   = tf.keras.metrics.Mean(name='cross_entropy_loss')
temporal_test_acc    = tf.keras.metrics.CategoricalAccuracy(name='temporal_test_acc')
source_train_acc     = tf.keras.metrics.CategoricalAccuracy(name='source_train_acc')
source_test_acc      = tf.keras.metrics.CategoricalAccuracy(name='source_test_acc')
office_test_acc      = tf.keras.metrics.CategoricalAccuracy(name='office_test_acc')
server_test_acc      = tf.keras.metrics.CategoricalAccuracy(name='server_test_acc')
conference_test_acc  = tf.keras.metrics.CategoricalAccuracy(name='conference_test_acc')

@tf.function
def test_step(images):
  logits, _ = classifier(images, training=False)
  return tf.nn.softmax(logits)

@tf.function
def train_clas_step(src_enc, src_labels, m, s):
  with tf.GradientTape() as tape:
    #Logits
    src_logits, _ = classifier(src_enc, training=True)
    src_logits_am = AM_logits(labels=src_labels, logits=src_logits, m=m, s=s)
    
    #Loss
    batch_cross_entropy_loss  = get_cross_entropy_loss(labels=src_labels,
                                                       logits=src_logits_am)
    total_loss = batch_cross_entropy_loss
    
  clas_gradients = tape.gradient(total_loss, classifier.trainable_variables)
  clas_optimizer.apply_gradients(zip(clas_gradients, classifier.trainable_variables))

  source_train_acc(src_labels, tf.nn.softmax(src_logits))
  cross_entropy_loss(batch_cross_entropy_loss)

In [9]:
learning_rate  = tf.keras.optimizers.schedules.PolynomialDecay(init_lr,
                                                               decay_steps=5000,
                                                               end_learning_rate=init_lr*1e-2,
                                                               cycle=True)
classifier      = Classifier(clas_hidden, num_classes, activation_fn)
clas_optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate, beta_1 = 0.5)

summary_writer = tf.summary.create_file_writer(log_dir)

In [10]:
m_anneal = tf.Variable(0, dtype="float32")
for epoch in tqdm(range(epochs)):  
  m_anneal.assign(tf.minimum(m*(epoch/1000.0), m))
  for source_data in src_train_set:
    train_clas_step(source_data[0], source_data[1], s, m_anneal)

  for data in time_test_set:
    temporal_test_acc(test_step(data[0]), data[1])

  for data in src_test_set:
    source_test_acc(test_step(data[0]), data[1])

  for data in office_test_set:
    office_test_acc(test_step(data[0]), data[1])

  for data in server_test_set:
    server_test_acc(test_step(data[0]), data[1])

  for data in conf_test_set:
    conference_test_acc(test_step(data[0]), data[1])

  with summary_writer.as_default():
    tf.summary.scalar("cross_entropy_loss", cross_entropy_loss.result(), step=epoch)
    tf.summary.scalar("temporal_test_acc", temporal_test_acc.result(), step=epoch)
    tf.summary.scalar("source_train_acc", source_train_acc.result(), step=epoch)
    tf.summary.scalar("source_test_acc", source_test_acc.result(), step=epoch)
    tf.summary.scalar("office_test_acc", office_test_acc.result(), step=epoch)
    tf.summary.scalar("server_test_acc", server_test_acc.result(), step=epoch)
    tf.summary.scalar("conference_test_acc", conference_test_acc.result(), step=epoch)

  cross_entropy_loss.reset_states()
  temporal_test_acc.reset_states()
  source_train_acc.reset_states()
  source_test_acc.reset_states()
  office_test_acc.reset_states()
  server_test_acc.reset_states()
  conference_test_acc.reset_states()

100%|██████████| 250/250 [00:58<00:00,  4.25it/s]


In [11]:
init_lr        = 0.0001
batch_size     = 32

learning_rate  = tf.keras.optimizers.schedules.PolynomialDecay(init_lr,
                                                               decay_steps=5000,
                                                               end_learning_rate=init_lr*1e-2,
                                                               cycle=True)

mlp = MLP([512, num_features], activation_fn)
mlp_optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate, beta_1 = 0.5)

notes           = "svd_class_batch"
log_data = "classes-{}_bs-{}_train_src_days-{}_train_trg_days-{}_train_trgenv_days-{}_alpha-{}_initlr-{}_num_feat-{}_act_fn-{}_{}".format(num_classes,
                                                                                                                                 batch_size,
                                                                                                                                 train_src_days,
                                                                                                                                 train_trg_days,
                                                                                                                                 train_trg_env_days,
                                                                                                                                 alpha,
                                                                                                                                 init_lr,
                                                                                                                                 num_features,
                                                                                                                                 activation_fn,
                                                                                                                                 notes)
log_dir        = os.path.join(repo_path, 'logs/TEST/{}'.format(log_data))
summary_writer = tf.summary.create_file_writer(log_dir)

In [12]:
svd_batch_loss   = tf.keras.metrics.Mean(name='svd_batch_loss')
svd_class_loss   = tf.keras.metrics.Mean(name='svd_class_loss')
server_train_acc = tf.keras.metrics.CategoricalAccuracy(name='server_train_acc')


@tf.function
def test_mlp_step(images):
  logits, _ = classifier(mlp(images, training=False), training=False)
  return tf.nn.softmax(logits)


@tf.function
def train_mlp_step(src_enc_class, src_enc, ser_enc_class, ser_enc, ser_labels):
  with tf.GradientTape() as tape:
    # batch stats
    ser_enc_mlp     = mlp(ser_enc, training=True)        
    svd_batch_align_loss = tf.math.reduce_mean(tf.math.abs(tf.math.reduce_mean(ser_enc_mlp, axis=0)-tf.math.reduce_mean(src_enc, axis=0)))
        
    # class stats
    ser_enc_mlp     = mlp(ser_enc_class, training=True)
    svd_class_align_loss = tf.math.reduce_mean(tf.math.abs(tf.math.reduce_mean(ser_enc_mlp, axis=0)-tf.math.reduce_mean(src_enc_class, axis=0)))
    
    total_loss = svd_class_align_loss + svd_batch_align_loss
    
    ser_logits, _ = classifier(mlp(ser_enc, training=False), training=False)

  mlp_gradients = tape.gradient(total_loss, mlp.trainable_variables)
  mlp_optimizer.apply_gradients(zip(mlp_gradients, mlp.trainable_variables))

  server_train_acc(ser_labels, tf.nn.softmax(ser_logits))
  svd_batch_loss(svd_batch_align_loss)
  svd_class_loss(svd_class_align_loss)
  
  
@tf.function
def train_mlp_test_step(src_enc_class, src_enc, ser_enc_class, ser_enc, ser_labels):
  with tf.GradientTape() as tape:
    # batch stats
    ser_enc_mlp     = mlp(ser_enc, training=True)
    _, src_u, src_v = tf.linalg.svd(src_enc) 
    _, ser_u, ser_v = tf.linalg.svd(ser_enc_mlp) 
        
    svd_batch_align_loss = tf.math.reduce_sum(tf.math.abs(tf.einsum('ij,ij->i', src_u, ser_u)))
        
    # class stats
    ser_enc_mlp     = mlp(ser_enc_class, training=True)
    s, src_u, src_v = tf.linalg.svd(src_enc_class) 
    _, ser_u, ser_v = tf.linalg.svd(ser_enc_mlp) 

    svd_class_align_loss = tf.math.reduce_sum(tf.math.abs(tf.einsum('ij,ij->i', src_u, ser_u)))
    
    total_loss = svd_class_align_loss + svd_batch_align_loss
    
    ser_logits, _ = classifier(mlp(ser_enc, training=False), training=False)

  mlp_gradients = tape.gradient(total_loss, mlp.trainable_variables)
  mlp_optimizer.apply_gradients(zip(mlp_gradients, mlp.trainable_variables))

  server_train_acc(ser_labels, tf.nn.softmax(ser_logits))
  svd_batch_loss(svd_batch_align_loss)
  svd_class_loss(svd_class_align_loss)

In [13]:
y_train_server_soft = get_target_labels(X_train_server, y_train_server)

def get_class_samples(X, y):
  tmp = X[np.where(np.argmax(y, axis=-1)==np.random.randint(0, num_classes))]
  np.random.shuffle(tmp)
  return tmp[:batch_size]

0.9710467706013363


In [14]:
for epoch in tqdm(range(epochs)):    
  for source_data, server_data in zip(src_train_set, server_train_set):
    train_mlp_step(get_class_samples(X_train_src, y_train_src), 
                   source_data[0], 
                   get_class_samples(X_train_server, y_train_server_soft),
                   server_data[0],
                   server_data[1])

  for data in time_test_set:
    temporal_test_acc(test_mlp_step(data[0]), data[1])

  for data in src_test_set:
    source_test_acc(test_mlp_step(data[0]), data[1])

  for data in office_test_set:
    office_test_acc(test_mlp_step(data[0]), data[1])

  for data in server_test_set:
    server_test_acc(test_mlp_step(data[0]), data[1])

  for data in conf_test_set:
    conference_test_acc(test_mlp_step(data[0]), data[1])

  with summary_writer.as_default():
    tf.summary.scalar("server_train_acc", server_train_acc.result(), step=epoch)
    tf.summary.scalar("svd_batch_loss", svd_batch_loss.result(), step=epoch)
    tf.summary.scalar("svd_class_loss", svd_class_loss.result(), step=epoch)

  server_train_acc.reset_states()
  svd_batch_loss.reset_states()
  svd_class_loss.reset_states()

100%|██████████| 250/250 [01:08<00:00,  3.64it/s]


In [ ]:
@tf.function
def predict(images):
  logits, fc1 = classifier(images, training=False)
  return tf.nn.softmax(logits), fc1

In [ ]:
def get_encodings(data):
  data_list = []
  for i in range(len(data)):
    _, encodings = predict(np.expand_dims(data[i], axis=0))
    data_list.append(np.array(encodings))
  return np.squeeze(data_list)

In [ ]:
def get_target_labels(source_data, source_labels, target_data, target_labels):
    centers = []
    for i in range(num_classes):
        centers.append(source_data[np.where(np.argmax(source_labels, axis=1)==i)].mean(axis=0))
    centers = np.array(centers) 

    kmeans = KMeans(n_clusters=9, init=centers, n_init=1).fit(target_data)
    print((np.argmax(target_labels, axis=1)==kmeans.labels_).mean())
    print(kmeans.labels_)
    return tf.one_hot(kmeans.labels_, num_classes)

In [ ]:
new_embed_ser = get_encodings(X_train_server)
new_embed_src = get_encodings(X_train_src)
_ = get_target_labels(new_embed_src, y_train_src, new_embed_ser, y_train_server)